In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/CLASE_40/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Clasificacion: canjeo de coupones de descuento

Trabajaremos con el dataset preprocesado en la notebook 'opcional_preprocesamiento_datos_clase'.
El mismo cuenta con información de distintas campañas de marketing en donde se ofrecen coupnes de descuento sobre distintas marcas y productos. El dataset preprocesado cuenta con fetures con información sobre las campañas, sobre los consumidores (hábitos de consumo y características demográficas) y sobre los items alcanzados por las promociones.

La variable target es el estado de canje de los cupones: "redemption_status". 

El dataset original está [acá](https://www.kaggle.com/vasudeva009/coupon-redemption-smote-feature-selection/data). Se aconseja mirar la notebook de preprocesamiento para entender mejor las variables del dataset.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTENC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif,RFECV,RFE
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,plot_confusion_matrix,roc_auc_score, classification_report, confusion_matrix, precision_recall_curve, auc

In [3]:
data=pd.read_csv('../Data/marketing/data_preprocessed.csv')
display(data.head(3))
data.dtypes

,Unnamed: 0,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,duration,month,year,...,mean_selling_price_cust,mean_other_discount_cust,mean_coupon_discount_cust,mean_quantity_item_coupon,mean_selling_price_item_coupon,mean_other_discount_item_coupon,mean_coupon_discount_item_coupon,brand,brand_type,category
0,0,1,13,27,1053,0,X,47,5,2013,...,184.260484,-33.168935,-0.287258,1.169768,84.478583,-51.732305,-1.241694,1105,Established,Grocery
1,1,2,13,116,48,0,X,47,5,2013,...,234.247013,-27.699169,-3.215039,1.050926,63.833519,-10.661343,0.000000,56,Local,Grocery
2,2,6,9,635,205,0,Y,32,3,2013,...,121.094495,-17.795660,-2.212082,1.353802,332.128380,-142.498493,-2.987321,560,Established,Pharmaceutical


Unnamed: 0                            int64
id                                    int64
campaign_id                           int64
coupon_id                             int64
customer_id                           int64
redemption_status                     int64
campaign_type                        object
duration                              int64
month                                 int64
year                                  int64
age_range                            object
marital_status                       object
rented                                int64
family_size                          object
no_of_children                       object
income_bracket                        int64
mean_quantity_cust                  float64
mean_selling_price_cust             float64
mean_other_discount_cust            float64
mean_coupon_discount_cust           float64
mean_quantity_item_coupon           float64
mean_selling_price_item_coupon      float64
mean_other_discount_item_coupon 

In [4]:
data=data.drop(['Unnamed: 0','id','customer_id','campaign_id','coupon_id'],axis=1)

In [5]:
categorical=['campaign_type','month','year','age_range','marital_status','rented','family_size','no_of_children',\
             'income_bracket','brand','brand_type','category']

data[categorical]=data[categorical].astype('category')
data.dtypes

redemption_status                      int64
campaign_type                       category
duration                               int64
month                               category
year                                category
age_range                           category
marital_status                      category
rented                              category
family_size                         category
no_of_children                      category
income_bracket                      category
mean_quantity_cust                   float64
mean_selling_price_cust              float64
mean_other_discount_cust             float64
mean_coupon_discount_cust            float64
mean_quantity_item_coupon            float64
mean_selling_price_item_coupon       float64
mean_other_discount_item_coupon      float64
mean_coupon_discount_item_coupon     float64
brand                               category
brand_type                          category
category                            category
dtype: obj

### Primera parte
* Ver el balance de clases en el dataset conun value counts de la columna redemption_status
* Hacer un train-test-split estratificado
* Convertir las variables categoricas a dummy. Conviene que nos quedemos con una versión no dummy del dataset de entrenamiento puesto que luego usaremos SMOTENC
* Opcional (recomendado): Armar una función que tome como input un modelo entrenado y un dataset de testeo e imprima las métricas más importantes para evaluar clasificación (classification_report, matriz de confución, area bajo las curvas ROC y Precision-Recall)
* Instanciar un modelo Random Forest entrenarlo y evaluarlo en el dataset de testeo

### Resampling de las clases

* Hacer un undersampling de la clase mayoritaria en el training set para balancear las clases. ¿De qué tamaño quedó el dataset de entrenamiento?
* Volver a entrenar un random forest y evaluarlo en test
* Repetir el procedimiento usando SMOTENC para sobresamplear la clase minoritaria
* Instanciar un Random Forest usando class_weight='balanced_subsample', entrenarlo en el training set original (sin resampling) y evaluarlo en test set

* Combinar las tres estrategias anteriores:
- Undersampling
- Oversampling
- Class weight
Usar una combinación de undersampling y oversampling que les parezca razonable. Idealmente habría que optimizar esta combinación mediante cross-validation, pero no hay que subestimar el tiempo de cómputo. Utilicen una combinación de sampling_stategy que de por resultado un dataset de tamaño reducido respecto del original. Usaremos ese dataset para hacer feature selection en la segunda parte

### Segunda Parte: Selección de Features

#### SelectKBest

Primero seleccionemos featuers usando SelectKbest tomando como medida de score el criterio 'f_classif'

* Hacer un pipeline que concatene el selector de features y un RandomForestClassifier con class_weight='balanced_subsample'
* Hacer una gridsearchCV variando el parámetro k del selector de features. Usar scoring='f1'.
* Graficar los resultados del procedimiento de cross-validation: scores de clasificacion vs nro de features
* ¿Con cuántas features se quedarían? ¿Cuáles?
* ¿Cuál es el score en el test set al seleccionar el subset de features elegido?
* Graficar la importancia de las features para el modelo (atributo feature_importance_)

#### Seleccion de Features por RFE
* Implementar RFECV usando como modelo un randomforest con clases pesadas. Usar un step grande (50) para tener una primera aproximacion
* Graficar los scores obtenidos para cada numero de features
* Usar RFE para seleccionar el número de features más adecuado. En este caso implementar un paso más fino (step=10), entrenando en todo el dataset de entremiento (sin cross-validation)
* Evaluar el modelo en el test set y mirar las feature_importance_